In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
np.random.seed(1)
categorías=['Guadalajara','Zapopan','Ciudad de México','Monterrey','Morelia','Mexicali']
ponderación=[1/4,1/5,1/4,7/40,1/20,3/40]
limites=[(3500,6000),(3000,12000),(4000,18000),(3000,6000),(2000,5000),(3500,5000)]
ciudad=np.random.choice(categorías,size=20,p=ponderación)
rentas=[]
for c in ciudad:
  rentas.append(np.random.randint(*limites[categorías.index(c)]))
df=pd.DataFrame()
df['Rentas']=rentas
df['Ciudad']=ciudad
df.head()

,Rentas,Ciudad
0,9285,Zapopan
1,5025,Monterrey
2,4531,Guadalajara
3,7415,Zapopan
4,4837,Guadalajara


# One-Hot

In [ ]:
pd.get_dummies(df) # Codifica todas las variables categóricas

In [ ]:
pd.get_dummies(df['Ciudad'])

In [7]:
OH=pd.get_dummies(df,columns=['Ciudad'],prefix='',prefix_sep='') # Codifica 'Ciudad' y me regresa el resto del DF
OH.head()

,Rentas,Ciudad de México,Guadalajara,Monterrey,Morelia,Zapopan
0,9285,0,0,0,0,1
1,5025,0,0,1,0,0
2,4531,0,1,0,0,0
3,7415,0,0,0,0,1
4,4837,0,1,0,0,0


# Codificación Ficticia

In [9]:
F=pd.get_dummies(df,prefix='',prefix_sep='',drop_first=True)
F.head()

,Rentas,Guadalajara,Monterrey,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,0,1,0,0
2,4531,1,0,0,0
3,7415,0,0,0,1
4,4837,1,0,0,0


In [12]:
# Categoría de referencia
CR,=set(OH.columns)-set(F.columns)
CR

'Ciudad de México'

# Codificación de efectos

In [14]:
E=pd.get_dummies(df,prefix='',prefix_sep='',drop_first=True)
E.head()

,Rentas,Guadalajara,Monterrey,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,0,1,0,0
2,4531,1,0,0,0
3,7415,0,0,0,1
4,4837,1,0,0,0


In [16]:
cr_idx,=np.where(df['Ciudad']==CR)
cr_idx

array([ 9, 11, 15, 17])

In [18]:
E.dtypes

Rentas         int64
Guadalajara    uint8
Monterrey      uint8
Morelia        uint8
Zapopan        uint8
dtype: object

In [19]:
E=E.astype('int')
E.dtypes

Rentas         int64
Guadalajara    int64
Monterrey      int64
Morelia        int64
Zapopan        int64
dtype: object

In [ ]:
E.loc[cr_idx,1:]=-1
E

# Regresión lineal

In [21]:
#@title One-Hot
linOH=LinearRegression()
linOH.fit(OH.drop(columns='Rentas'),OH['Rentas'])

LinearRegression()

In [22]:
linOH.coef_

array([ 4525.88333333, -1799.11666667, -1357.36666667, -3287.36666667,
        1917.96666667])

In [23]:
linOH.intercept_

6382.366666666667

In [24]:
p=df.groupby('Ciudad').mean()
p

,Rentas
Ciudad,
Ciudad de México,10908.250000
Guadalajara,4583.250000
Monterrey,5025.000000
Morelia,3095.000000
Zapopan,8300.333333


In [25]:
p.mean()

Rentas    6382.366667
dtype: float64

El promedio de la variable `p` corresponde a la intersección en la regresión lineal.

In [26]:
p-p.mean()

,Rentas
Ciudad,
Ciudad de México,4525.883333
Guadalajara,-1799.116667
Monterrey,-1357.366667
Morelia,-3287.366667
Zapopan,1917.966667


In [27]:
linOH.coef_

array([ 4525.88333333, -1799.11666667, -1357.36666667, -3287.36666667,
        1917.96666667])

Los coeficientes resultantes de la regresión lineal corresponden a la diferencia entre el promedio de la renta por ciudad y el promedio general.

In [28]:
#@title Codificación ficticia
linF=LinearRegression()
linF.fit(F.drop(columns='Rentas'),F.Rentas)

LinearRegression()

In [29]:
linF.coef_

array([-6325.        , -5883.25      , -7813.25      , -2607.91666667])

In [30]:
linF.intercept_

10908.250000000002

In [31]:
p.loc[CR] # Promedio de la categoría de referencia

Rentas    10908.25
Name: Ciudad de México, dtype: float64

En la codificación ficticia, al hacer una regresión lineal, la intersección representa el promedio de la categoría de referencia.

In [32]:
p-p.loc[CR]

,Rentas
Ciudad,
Ciudad de México,0.000000
Guadalajara,-6325.000000
Monterrey,-5883.250000
Morelia,-7813.250000
Zapopan,-2607.916667


In [33]:
linF.coef_

array([-6325.        , -5883.25      , -7813.25      , -2607.91666667])

Los coeficientes corresponden a la diferencia entre los promedios de las rentas por ciudad y la intersección.

In [34]:
#@title Codificación de Efectos
linE=LinearRegression()
linE.fit(E.drop(columns='Rentas'),E.Rentas)

LinearRegression()

In [35]:
linE.coef_

array([-1799.11666667, -1357.36666667, -3287.36666667,  1917.96666667])

In [37]:
linOH.coef_[1:]

array([-1799.11666667, -1357.36666667, -3287.36666667,  1917.96666667])

In [38]:
linE.intercept_

6382.366666666668

In [39]:
linOH.intercept_

6382.366666666667

In [40]:
linE.intercept_-linE.coef_.sum()

10908.25

In [41]:
p.loc[CR]

Rentas    10908.25
Name: Ciudad de México, dtype: float64